# Obiettivo del progetto

L'obiettivo del progetto è estendere il simulatore 2D di robot realizzato per il progetto di **Paradigmi di Programmazione e Sviluppo** per dotarlo di comportamenti appresi tramite tecniche di Reinforcement Learning, in particolare Q-learning e Deep Q-learning.

Si è proposto di realizzare i seguenti tre comportamenti:
- [**phototaxis**](./phototaxis.ipynb): movimento verso una sorgente luminosa;
- [**obstacle avoidance**](./obstacle-avoidance.ipynb): movimento con evitamento di ostacoli;
- [**exploration**](./exploration.ipynb): massimizzare l’esplorazione dell’ambiente evitando le collisioni.

# Vincoli

I vincoli posti sono:

- ambiente: griglie 5×5, 20×20 e 30×30 m;
- sensori: 8 prossimità + 8 luce + posizione + orientazione;
- luci con raggio 0.2 m e irradiazione 5 m;
- ostacoli rettangolari con dimensioni variabili.

# Starting point
Siamo partiti da un simulatore in grado di gestire un ambiente composto da N robot aventi un comportamento specificato da configurazione.

Nell'ambiente sono presenti entità statiche, come ostacoli e luci e entità dinamiche, come altri robot.
I robot sono in grado di percepire le altre entità attraverso i sensori di prossimità e di luce e possono interagire nell'ambiente attraverso gli attuatori tramite comportamenti pre-programmati. Il movimento dei robot segue un movimento differenziale in cui posono essere applicate delle velocità diverse alle due ruote.

## Configurazione del simulatore (yaml)

Per configurare il simulatore si possono utilizzare o file `yaml` oppure la GUI dedicata.

L'utilizzo di file `yaml` permette di configurare il simulatore in modo semplice ed efficace. Un esempio di struttura è la seguente:

```yaml
simulation:
  seed: 42
environment:
  width: 12
  height: 10
  entities:
    - light:
        position: [10, 5]
        illuminationRadius: 6.0
        intensity: 1.0
        radius: 0.2
        attenuation: 1.0
    - obstacle:
        position: [6, 5]
        orientation: 0.0
        width: 2.0
        height: 6.0
    - robot:
        position: [2, 2]
        orientation: 45.0
        radius: 0.25
        speed: 1.0
        withProximitySensors: true
        withLightSensors: true
        behavior: Phototaxis
```

I file di configurazione sono stati utilizzati durante l'addestramento e la valutazione degli agenti.

## Adattamento e modellazione del simulatore

Il simulatore è stato esteso per integrare comportamenti basati su tecniche di Q-learning e Deep Q-learning rendendo i robot autonomi.
Il simulatore è implementato interamente in `Scala`, mentre la parte di Reinforcement Learning è sviluppata in `Python`.

### Comunicazione python-simulatore

Per il collegamento tra i due linguaggi è stato adottato `gRPC`. Inoltre, è stato definita un’interfaccia di interazione modellata sullo stile di `PettingZoo`, così da mantenere coerenza con le principali librerie di Reinforcement Learning multi-agente.

---

### Simulatore lato Scala

Di seguito viene fornita una descrizione delle modifiche apportate implementativamente al simulatore.

#### Agente

Invece che apportare delle modifiche ai robot esistenti si è preferito creare delle entità ad-hoc: gli agenti.
Nel simulatore in Scala, l'agente ha le stesse caratteristiche del robot ad eccezione del `Behavior` in cui non associamo più un comportamento programmatico ma adesso è composto da `Reward`, `Termination` e `Truncation`.

#### Reward

La `Reward` è una funzione associata all'agente che permette di osservare lo stato dell'ambiente (precedente e corrente) e calcolare una ricompensa (bonus o penalità) adeguata al task che deve risolvere.

Signature per la reward:

```scala
def evaluate(prev: BaseState, current: BaseState, entity: Agent, action: Action[?]): Double
```

Non sempre le informazioni dello stato precedente e corrente sono sufficienti per calcolare una ricompensa ottimale per il task in esame.
È stata quindi introdotta una `StatefulReward` che permette di salvare informazioni da riutilizzare negli step successivi.

Questa tipologia di reward è risultata particolarmente utile nel task di **Exploration** in quanto permette di salvare informazioni utili, tra cui le posizioni già visitate dagli agenti.

#### Termination

La `Termination` permette di concludere in maniera naturale la finestra di addestramento sia come situazione positiva o negativa, indipendentemente del numero massimo di step, se definiti.
È stata modellata sulla base del task specifico da addestrare.

La signature per la termination viene definita come:

```scala
def evaluate(prev: BaseState, current: BaseState, entity: Agent, action: Action[?]): Boolean
```

Come nel caso della `StatefulReward` si è reso necessario aggiungere anche il concetto di `StatefulTermination`, permettendo ad esempio nel task di **Exploration** di terminare il processo quando l'agente ha visitato una certa percentuale dell'ambiente.

#### Truncation

La `Truncation` è stata realizzata in modo equivalente alla `Termination`. Gestisce invece le condizioni di fallimento forzato.
Il limite massimo di step è gestito principalmente lato `Python` per questo motivo non è stata utilizzata.

---

### Lato Python

In `Python` è stato creato l'agente, l'ambiente osservabile da esso e gli algoritmi di training e valutazione.

#### Descrizione dell'ambiente

L'ambiente funziona da ponte con il simulatore e permette all'agente di interagire con esso svolgendo azioni e di percepire il mondo a lui circostante tramite le osservazioni.
È stato modellato sulla base della libreria `gymnasium`, contiene quindi i seguenti metodi:

- step:

  ```python
  def step(self, actions: dict) -> tuple[dict, dict, dict, dict, dict]
  ```
  
  fa svolgere le azioni fornite agli agenti, effettuando un `tick` nel simulatore e restituisce quindi, per ogni agente:

  - osservazioni;
  - ricompense;
  - terminazioni;
  - troncamenti;
  - informazioni aggiuntive.
  
- render:

  ```python
  def render(self, width: int = 800, height: int = 600) -> np.ndarray
  ```

  ritorna un immagine in formato RGB delle dimensioni indicate, contenente una rappresentazione dello stato del simulatore al momento della chiamata.

- reset:

  ```python
  def reset(self, seed: int = 42) -> tuple[dict, dict]:
  ```

  riporta il simulatore alla configurazione iniziale e ritorna, per ogni agente:

  - osservazioni;
  - informazioni aggiuntive.

Sono presenti, inoltre, i metodi:

- `init`: utile a stabilire la connessione con il client e inizializzare l'ambiente con la configurazione;
- `close`: per chiudere la connessione con `gRPC`.

L'ambiente è stato creato come classe astratta `AbstractEnv` così che ciascuno potesse implementare il proprio modo per codificare e decodificare le osservazioni e azioni rispettivamente.

```python
@abstractmethod
def _encode_observation(
    self, proximity_values, light_values, position, orientation, visited_positions
):
    """Encode the observation from proximity, light values, position, orientation and visited_positions"""
    pass

@abstractmethod
def _decode_action(self, action):
    """Decode the action into the appropriate format"""
    pass
```

Le implementazioni specifiche dipendono da ciascun task, perciò verranno spiegate più nel dettaglio nei notebook referenziati successivamente.

#### Agenti in Python

In Python sono presenti due varianti dell'agente: il `QAgent` e il `DQAgent`.

##### QAgent

Il `QAgent` interagisce con un ambiente discreto e mantiene una Q-table inizializzata a zero. Le azioni vengono selezionate tramite politica epsilon-greedy: con probabilità ε si esplora casualmente, altrimenti si sceglie l’azione con il valore Q massimo. La Q-table viene aggiornata secondo la formula del Q-learning, considerando il fattore di apprendimento (α), il discount factor (γ) e la distinzione tra episodi terminati o troncati. Il tasso di esplorazione ε decresce esponenzialmente secondo la formula:

$$
\epsilon = \epsilon_{\min} + (\epsilon_{\max} - \epsilon_{\min}) \cdot e^{-\text{decay} \cdot \text{episode}}
$$

Ulteriori funzionalità consentono di salvare e caricare lo stato dell’agente (Q-table e parametri) tramite file `.npz`.

##### DQAgent

Il `DQAgent` è progettato per apprendere politiche ottimali in ambienti complessi utilizzando due **Deep Q-Network**: la rete principale (`action_model`), che stima i valori Q delle azioni, e la rete target (`target_model`), aggiornata periodicamente per rendere l’apprendimento più stabile. L’agente interagisce con l’ambiente tramite una strategia epsilon-greedy e memorizza le esperienze raccolte in una replay memory, così da poterle riutilizzare più volte durante l’addestramento e ridurre la correlazione tra i campioni.

Durante l’interazione con l’ambiente, le transizioni non vengono sempre memorizzate immediatamente. L’agente può infatti accumulare più passi consecutivi prima di salvarli nella replay memory, calcolando un ritorno che tiene conto di più ricompense future. Questo meccanismo, noto come **n-step return**, consente una propagazione più rapida dell’informazione sulle ricompense e rende il processo di apprendimento più stabile.

Il tasso di esplorazione ε decresce esponenzialmente nel corso degli episodi, seguendo la stessa formulazione adottata per il `QAgent`, permettendo un passaggio graduale dall’esplorazione iniziale allo sfruttamento delle azioni apprese.

Funzionalità principali del DQAgent:

- **Selezione dell’azione (`choose_action`)**
  L’agente sceglie le azioni secondo una politica epsilon-greedy, esplorando casualmente con probabilità ε o selezionando l’azione con il valore Q più elevato stimato dalla rete principale.

- **Memorizzazione delle transizioni (`store_transition`)**  
  Le esperienze vengono raccolte e, dopo aver eventualmente accumulato più passi consecutivi, salvate nella replay memory sotto forma di transizioni arricchite da ritorni multi-step.

- **Campionamento mini-batch (`get_random_batch` e `get_random_batch_from_replay_memory`)**  
  Dalla replay memory vengono estratti mini-batch casuali, riducendo la correlazione tra esempi e migliorando la stabilità dell’addestramento.

- **Aggiornamento della rete principale (`dqn_update`)**
  I pesi della rete principale vengono aggiornati minimizzando la perdita TD (Temporal Difference), calcolata confrontando i valori Q stimati con i target ottenuti dalle transizioni memorizzate.

- **Aggiornamento della rete target (`update_target_model`)**  
  La rete target viene sincronizzata periodicamente con la rete principale per limitare oscillazioni e instabilità durante l’apprendimento.

- **Decadimento del tasso di esplorazione (`decay_epsilon`)**  
  Il valore di ε diminuisce progressivamente nel tempo, favorendo lo sfruttamento delle politiche apprese.

- **Inizializzazione della memoria (`simple_dqn_replay_memory_init`)**  
  Prima dell’addestramento, la replay memory viene popolata con transizioni casuali per garantire un avvio stabile del processo di apprendimento.

- **Calcolo della perdita TD singola (`compute_td_loss`)**  
  Consente di valutare l’errore TD su singole transizioni, risultando utile per analisi e debugging.

- **Salvataggio e caricamento (`save` e `load`)**  
  Permette di salvare e ripristinare modelli, parametri e stato dell’agente, rendendo possibile interrompere e riprendere l’addestramento senza perdita di informazioni.

###### Deep Q-Network

La `DQNetwork` è la rete neurale utilizzata per stimare i valori Q delle azioni in un dato stato, permettendo all’agente di apprendere politiche ottimali anche in ambienti complessi e a spazi di stato continui.

- **Input:** vettore di caratteristiche dello stato (`input_count`).  
- **Hidden layers:** uno o più strati densi con attivazione ReLU; numero di neuroni definito da `neuron_count_per_hidden_layer`.  
- **Output:** uno strato con un neurone per ciascuna azione (`action_count`), che produce i valori Q.  
- **Ottimizzazione e perdita:** loss **MSE** e ottimizzatore **Adam** per aggiornamenti stabili.  

Funzionalità principali: **predizione valori Q (`predict`)**, **aggiornamento pesi (`update_weights`)** con la rete target e possibilità di visualizzare la struttura della rete tramite summary o diagrammi.

---

# Metodo risolutivo

Per la realizzazione dei task si è seguito un processo iterativo.
Inizialmente sono stati generati gli ambienti di training e test, successivamente vengono effettuati gli addestramenti e le valutazioni.

Sulla base dei risultati ottenuti dai primi training, per un numero di epoche più ridotto, vengono analizzati gli errori che gli agenti compiono; queste analisi permettono iterativamente di modificare l'approccio risolutivo, la _reward_ e le funzioni di _truncation_ e _termination_ in modo che meglio riflettano i requisiti del task specifico.

Si è prima realizzata l'implementazione riguardante il Q-Learning tabellare, e solo successivamente il Deep-Q-Learning.

Affrontare inizialmente i task con i `QAgent` ha permesso di individuare rapidamente le criticità ricorrenti legate alla funzione di reward, che rendevano l’addestramento poco efficace. Successivamente, l’addestramento dei `DQAgent` è risultato più agevole, grazie alla conoscenza preventiva delle strategie per aggirare i problemi più comuni.

Di seguito sono spiegate tutte le fasi principali necessarie alla risoluzione dei task.

## Generazione degli ambienti

Per la generazione degli ambienti di training e test è stato utilizzato lo script `generate_environments.py`([link al file](../../scripts/generate_environments.py)).

Lo script permette di configurare parametri riguardanti:

- **dimensioni ambiente**: `--width` e `--height`
- **ostacoli**:
  - Numero: `--obstacle-min-num` e `--obstacle-max-num`;
  - Dimensioni: `--obstacle-min-size` e `--obstacle-max-size`;
- **luci**:
  - Numero: `--light-min-num` e `--light-max-num`;
  - Parametri di illuminazione automatici.

Esempio di utilizzo:

```bash
python generate_environments.py \
  --num 10 \
  --config-root resources generated obstacle-avoidance \
  --width 10 --height 10 \
  --obstacle-min-num 4 --obstacle-max-num 15 \
  --obstacle-min-size 0.5 --obstacle-max-size 8.0 \
  --light-min-num 0 --light-max-num 0
```

Lo script genera file `yaml` validi controllando la validità dell'ambiente tramite il simulatore. Utilizza la libreria `environment_generator.py`([link al file](../../scripts/lib/environment_generator.py)), la quale verifica che gli ambienti generati siano validi.

## Training

La fase di addestramento utilizza due approcci differenti in base alla complessità del task: **Q-learning tabulare** e
**Deep Q-learning**.
Il training viene eseguito tramite script `Python` dedicati (`train-qagent.py` e `train-dqagent.py`), che si occupano di
gestire l’interazione tra agente e simulatore tramite `gRPC`, l’esecuzione degli episodi e il salvataggio dei dati
necessari alla successiva fase di valutazione.

Gli script permettono inoltre di configurare i principali parametri di training (numero di episodi, step massimi,
frequenza dei checkpoint) e di riprendere un addestramento precedente, rendendo gli esperimenti ripetibili e
controllabili.

### Q-learning

L’addestramento con **Q-learning** è utilizzato per task con spazio degli stati discreto e di dimensione limitata.  
In questo caso l’agente mantiene una **Q-table**, che associa ad ogni coppia stato–azione un valore di utilità appreso
durante l’interazione con l’ambiente.

Durante il training l’agente segue una politica **epsilon-greedy**, che bilancia esplorazione e sfruttamento:
inizialmente vengono provate molte azioni diverse, mentre con il progredire degli episodi l’agente tende a sfruttare le
azioni che hanno prodotto le ricompense migliori.
Il parametro `epsilon` viene quindi ridotto progressivamente tramite decadimento esponenziale.

Lo script di training gestisce:

- l’esecuzione degli episodi di addestramento;
- l’aggiornamento della Q-table in base alle ricompense osservate;
- il salvataggio periodico della Q-table e dei parametri dell’agente tramite checkpoint.

### Deep Q-learning

Per task più complessi, caratterizzati da osservazioni continue o da uno spazio degli stati più ampio, viene utilizzato
**Deep Q-learning**.  
In questo caso la funzione Q non è rappresentata da una tabella, ma viene approssimata tramite una **rete neurale**.

Durante il training l’agente memorizza le esperienze sotto forma di transizioni
`(stato, azione, ricompensa, stato successivo)` all’interno di una **replay memory**.  
Le reti neurali vengono aggiornate campionando mini-batch casuali da questa memoria, riducendo la correlazione tra
esperienze consecutive e rendendo l’apprendimento più stabile.

Per migliorare ulteriormente la stabilità, il training utilizza due reti distinte:

- una **action network**, aggiornata frequentemente;
- una **target network**, aggiornata periodicamente.

Anche in questo caso l’agente utilizza una politica **epsilon-greedy**, con una riduzione graduale dell’esplorazione nel
corso del training.
La convergenza viene monitorata tramite la media mobile delle reward e, se questa supera una soglia prefissata, il
training viene terminato anticipatamente (**early stopping**).

Durante l’addestramento vengono salvati checkpoint periodici contenenti i pesi delle reti neurali e i parametri
dell’agente, che vengono poi utilizzati nella fase di valutazione.

## Evaluation

La valutazione delle performance dei `QAgent` e `DQAgent` viene effettuata tramite la funzione `evaluate`, la cui signature è la seguente:

```python
def evaluate(
    env: PhototaxisEnv | ObstacleAvoidanceEnv | ExplorationEnv,
    agents: dict[str, QAgent | DQAgent],
    configs: list[str],
    max_steps: int,
    did_succeed: Callable[[float, bool, bool], bool],
    window_size: int = 100,
) -> dict
```

Viene passato l'`environment` specifico per il task, oltre a un dizionario con gli agenti e i relativi id.
Questi vengono valutati su tutte le configurazioni fornite in `configs` per un numero di passi pari a `max_steps`.
La _lambda_ `did_succeed` permette di capire se l'agente ha terminato l'episodio con successo o fallimento.
Il parametro `window_size` permette di calcolare la `moving average reward`.

La funzione supporta nativamente scenari **multi-agente** (da 1 a $N$) utilizzando dizionari per gestire stati e azioni indipendenti, isolando correttamente gli agenti che terminano l'episodio prima degli altri. Durante la valutazione la policy è deterministica (_greedy_).

Le metriche restituite sono essenziali per diagnosticare la qualità dell'apprendimento:

- **`success_rate`** e **`successes_idx`**: indicano la robustezza dell'agente e quali specifiche configurazioni riesce a risolvere;
- **`median_steps_to_success`**: misura l'efficienza (velocità) nel raggiungere l'obiettivo;
- **`total_rewards`** e **`moving_avg_reward`**: valutano la performance cumulativa e la stabilità del comportamento durante l'episodio;
- **`td_losses`** (per `DQAgent`): monitora l'errore di stima dei valori $Q$, utile per rilevare incertezze o problemi di generalizzazione su stati non visti in training.



# Per eseguire i notebook relativi ai task

Per eseguire il simulatore è necessario installare le dipendenze python (`uv pip install -r pyproject.toml`) e generare le definizioni di protobuf, quindi avviare il simulatore con docker.

In [ ]:
!cd ../../.. && ./create_proto_definitions.sh
!docker run ghcr.io/scala-robotics-simulator/pps-22-srs:v2.0.0
# Per poter eseguire i notebook relativi ai task è necessario avviare il simulatore prima

# Task

I task da svolgere sono:

- [**phototaxis**](./phototaxis.ipynb): generazione dell'agente in un punto casuale della mappa con l'obiettivo di raggiungere la prima luce disponibile cercando di evitare muri. In caso l'agente non rilevi valori con il sensore di luce deve entrare in uno stato di esplorazione.
- [**obstacle avoidance**](./obstacle-avoidance.ipynb): generazione dell'agente in un punto casuale dell'ambiente. L'obiettivo è quello di muoversi nello spazio senza toccare ostacoli e muri in un certo tempo definito.
- [**exploration**](./exploration.ipynb): generazione dell'agente in un punto casuale dell'ambiente. L'obiettivo è quello di massimizzare la copertura visitata dell'ambiente cercando di evitare gli ostacoli.

# Conclusioni

## Commenti finali

Il progetto ci ha permesso di esplorare tecniche di controllo robotico automatico, attraverso il **Reinforcement Learning**, in particolare _Q-learning_ e _Deep Q-learning_.

Realizzando interamente sia il simulatore che il sistema di comunicazione tra esso e gli agenti, abbiamo potuto sperimentare in un ambiente controllato l'efficacia delle tecniche di apprendimento automatico applicate al controllo robotico.

Tutti i task sono stati realizzati sia con tecniche di **Q-Learning** che **Deep Q-Learning**.
I task di *phototaxis* e *obstacle avoidance* sono risultati più semplici, mentre *exploration* è risultato un problema molto più complesso.

**Caratteristiche salienti del progetto:**

1. **architettura ibrida Scala-Python**: utilizzare `gRPC` per la comunicazione tra il simulatore e gli agenti ha permesso di sfruttare le potenzialità di entrambi i linguaggi; `Scala` per la robustezza, immutabilità e performance del simulatore, `Python` per il ricco ecosistema di librerie di machine learning e deep learning;

2. **reward engineering**: l'utilizzo di funzioni di ricompensa stateless e stateful ha permesso di realizzare funzioni adatte ai diversi scenari di apprendimento, migliorando la velocità di convergenza e la qualità delle politiche apprese;

3. **generazione ambienti**: lo script per generare ambienti casuali ha permesso di realizzare esperimenti su larga scala, testando la robustezza delle politiche apprese in ambienti diversi.

**Confronto tra Q-learning e Deep Q-learning:**

- **Q-learning**:
  - Vantaggi:
    - interpretabilità;
    - semplicità di implementazione;
    - bassa richiesta computazionale.
  - Svantaggi:
    - scalabilità limitata a spazi di stato piccoli;
    - difficoltà nell'estrazione di caratteristiche rilevanti dagli stati complessi.

- **Deep Q-learning**:
  - Vantaggi:
    - capacità di gestire spazi di stato complessi e di grandi dimensioni;
    - apprendimento automatico delle caratteristiche rilevanti dagli stati.
  - Svantaggi:
    - maggiore complessità di implementazione;
    - elevata richiesta computazionale;
    - reward engineering più complesso.


## Lavori futuri

Allo stato attuale il simulatore e gli agenti autonomi funzionano sufficientemente bene per compiti semplici, si potrebbe però ampliare il simulatore con:

- **hybrid control**: la possibilità di controllare sia in modo programmatico che tramite *Reinforcement Learning* gli agenti, permettendo quindi di ottenere performance migliori in compiti difficili da risolvere con tecniche di controllo autonomo;
- **cooperative multi agents**: il sistema, allo stato attuale, supporta solo la valutazione multi agente sui diversi task, ma non l'addestramento multi agente. Uno sviluppo interessante potrebbe essere la realizzazione di task cooperativi, quali ad esempio il *flocking* o il *clustering*.